In [4]:
 # boolean network packages
from sad2_final_project.boolean_bn import BN
from sad2_final_project.boolean_bn import simulate_trajectories_to_csv 
from sad2_final_project.analysis import BooleanNetworkExperiment
from boolean import BooleanAlgebra 
# system packages
import os
from pathlib import Path


# paths
## set global dir
cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())
## create paths 
DATA_PATH = Path('data')
# # TODO OBSOLETE 
# DATASET_PATH = DATA_PATH / 'datasets'
# BN_TRUE_PATH = DATA_PATH / 'bn_ground_truth'
# BN_DATASET_BNFINDER_FORMAT_PATH = DATA_PATH / 'datasets_bnfinder_format'
# BN_RESULT_PATH = DATA_PATH / "result"
## create directories
!mkdir {DATA_PATH}
# TODO OBSOLETE
# !mkdir -p {DATASET_PATH}
# !mkdir -p {BN_TRUE_PATH}
# !mkdir -p {BN_DATASET_BNFINDER_FORMAT_PATH}
# !mkdir -p {BN_RESULT_PATH}




/home/asia/rok3/sad2/final_project/SAD2_final_project
mkdir: cannot create directory ‘data’: File exists


## Part 1 

### Numerical analysis of Bnfinder
#### Task
Construct several Boolean networks with sizes (measured by the number of nodes or variables) ranging from 5 to 16.† Each node should have no more than three parent nodes, and the Boolean functions governing individual nodes should be generated at random.

#### Goal
The goal of this study is to determine how the type and amount of time-series data generated from Boolean network dynamics affect the accuracy of dynamic Bayesian network (DBN) structure inference. In particular, we aim to identify:
- how the presence of attractor states in trajectories influences reconstruction accuracy,
- how trajectory length, sampling frequency, and the number of trajectories affects model metrics
- how these effects depend on network size and update dynamics,
- which data-generation techniques yield stable and informative reconstructions. 

Because synchronous and asynchronous Boolean network updates correspond to different classes of stochastic processes, all results are analyzed separately for these two update modes. Furthermore, since different scoring functions (MDL and BDe) are not directly comparable in scale, they are treated as different experiment groups for same boolean networks. 

---

#### Sets

To ensure reproducibility and comparisons, experiments are organized into predefined sets.
##### Groups
1. **Network size groups**
   Networks are grouped by number of nodes (5–16) to analyze scaling behavior.
2. **Update mode groups**
   Synchronous and asynchronous updates are treated separately, as they correspond to different stochastic processes (deterministic map vs stochastic transition system).
3. **Scoring function groups**
   MDL and BDe are analyzed independently. Absolute score values are not compared across scoring functions; only trends with respect to accuracy are considered.
4. **Random function sets**
   For each experimental condition, multiple independently generated Boolean networks are used.
   Random seeds are fixed per network/experiment (TODO ?) instance so that different data-generation parameters (sampling frequency, trajectory length, number of trajectories) are evaluated on identical underlying networks.
   <!-- is not possible we just need to use seed to experiments are reproducle -->

---

##### Averaging and Distributions

All reported results are based on **distributions**, not single values.
Depending on the experiment, averaging is performed over:

- trajectories (within a dataset),
- independent datasets,
- independently generated networks.

The aggregation strategy is explicitly chosen for each experiment to match the source of variability under investigation.


<!-- ### Impact of Proportion of Attractor States in Trajectories

**Objective.**
To quantify how an increasing proportion of attractor states in trajectories affects the sensitivity of network structure reconstruction.

The central hypothesis is that a high proportion of attractor states leads to strong temporal autocorrelation, which **reduces the effective number of independent observations**, thereby decreasing the sensitivity (recall) of detected edges in the inferred network.

**Experimental design.**

- For each network, datasets are generated with controlled proportions of attractor states:
  (0%, 10%, \ldots, 100%).
- Trajectories are made sufficiently long to ensure that attractors are reached whenever the target proportion is nonzero.
- Sampling frequency is varied to contrast regimes of strong versus weak temporal dependence.

**Autocorrelation analysis.**
For each dataset, temporal dependence is quantified using the **autocorrelation function (ACF)**:
[
\rho(k) = \frac{\mathrm{Cov}(X_t, X_{t+k})}{\mathrm{Var}(X_t)}
]
computed separately for each node and then aggregated (mean or maximum across nodes).

From the ACF, the **effective sample size (ESS)** is estimated:
[
\mathrm{ESS} \approx \frac{N}{1 + 2\sum_{k=1}^{K} \rho(k)},
]
where (N) is the nominal number of observations and (K) is the truncation lag where autocorrelation becomes negligible.

Reconstruction accuracy is then analyzed as a function of ESS rather than nominal sample size.

**Rationale.**
This isolates the effect of attractor-induced redundancy and avoids conflating “amount of data” with “amount of information”. -->

---

#### Methodology

##### 1. Relation Between Trajectory Length and Entering Attractors

**Objective.**
To characterize how trajectory length is related to the probability of entering attractors as a function of network size and dynamics.

**Experimental design.**

- The target attractor proportion is **not controlled**; trajectories evolve naturally.
- Trajectory lengths are varied in increments proportional to network size:
    - from 5 steps to 50 by 5
    - from 50 steps to 200 by 10
- Networks are grouped by size (from 4 to 16 nodes, in steps of two).
- The number of parents per node is randomly chosen from set of $\{1,2,3\}$ to avoid conditioning results on a fixed connectivity pattern.

**Measured quantities.**

- Probability of reaching an attractor as a function of trajectory length.
- How different groups (below TODO - inner link) differ in in this probability.

**Rationale.**
Attractor entry is an emergent property of the dynamics. Controlling it directly is undesirable, as it would introduce selection bias. This experiment instead characterizes the **natural scaling behavior** of Boolean network dynamics.

In [ ]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    # paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',
    # experiment values
    num_nodes=[4, 6],
    update_mode=["synchronous"], # , "asynchronous"
    trajectory_length=list(range(50, 61, 10)),
    n_trajectories=[50],
    sampling_frequency=[1],
    score_functions=["MDL", "BDE"],
    # TODO - add metrics options 

    n_parents_per_node=[[1, 2, 3, 4]],
    # amount of samples per experiment 
    n_repetitions=2,
    simulate_trajectories_to_csv_kwargs = {
            # "sampling_frequency": 1,
            "target_attractor_ratio": 0.4,  # Approximate fraction of trajectory in attractor (0-1)
            "tolerance": 0.3,               # Allowed deviation from the calculated entrance step (0-1)
            "max_iter": 100,                 # Maximum attempts to generate a valid state per step before restarting
            "max_trajectory_restarts": 1000  # Maximum number of trajectory restarts allowed
        }
) 
exp_trajectory_length.show_experiment_df()

In [3]:
exp_trajectory_length.run_experiment()

,num_nodes,update_mode,trajectory_length,n_trajectories,sampling_frequency,score_function,n_parents_per_node,rep_id,condition_id
0,4,synchronous,50,50,1,MDL,"[1, 2, 3, 4]",0,0000
1,4,synchronous,50,50,1,MDL,"[1, 2, 3, 4]",1,0001
2,4,synchronous,50,50,1,BDE,"[1, 2, 3, 4]",0,0002
3,4,synchronous,50,50,1,BDE,"[1, 2, 3, 4]",1,0003
4,4,synchronous,60,50,1,MDL,"[1, 2, 3, 4]",0,0004
5,4,synchronous,60,50,1,MDL,"[1, 2, 3, 4]",1,0005
6,4,synchronous,60,50,1,BDE,"[1, 2, 3, 4]",0,0006
7,4,synchronous,60,50,1,BDE,"[1, 2, 3, 4]",1,0007
8,6,synchronous,50,50,1,MDL,"[1, 2, 3, 4]",0,0008
9,6,synchronous,50,50,1,MDL,"[1, 2, 3, 4]",1,0009


##### 2. Impact of Sampling Frequency

**Objective.**
To determine how temporal subsampling affects autocorrelation, effective sample size, and reconstruction accuracy.

Dynamic Bayesian network inference assumes conditional independence of observations given parent states in the previous time slice. Excessive temporal dependence violates this assumption in practice by introducing redundant observations.

**Experimental design.**

- For fixed networks and trajectory lengths, datasets are generated using multiple sampling frequencies (1, 2, 3, 4, 5).
- For each dataset
    - ACF and ESS are computed,
    - MDL and BDe scores are extracted from BNFinder2 logs,

**Analysis.**
Accuracy is analyzed jointly as a function of: 
<!-- What does it mean jointly ?? -->

* sampling frequency,
* ESS,
* scoring function (MDL or BDe).

**Rationale.**
This experiment identifies sampling regimes that balance reduced autocorrelation against loss of dynamic information due to over-subsampling.

---

In [ ]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    # paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',
    # experiment values
    num_nodes=[4, 6],
    update_mode=["synchronous"], # , "asynchronous"
    trajectory_length=list(range(50, 61, 10)),
    n_trajectories=[50],
    sampling_frequency=[1],
    score_functions=["MDL", "BDE"],
    # TODO - add metrics options 

    n_parents_per_node=[[1, 2, 3, 4]],
    # amount of samples per experiment 
    n_repetitions=2,
    simulate_trajectories_to_csv_kwargs = {
            # "sampling_frequency": 1,
            "target_attractor_ratio": 0.4,  # Approximate fraction of trajectory in attractor (0-1)
            "tolerance": 0.3,               # Allowed deviation from the calculated entrance step (0-1)
            "max_iter": 100,                 # Maximum attempts to generate a valid state per step before restarting
            "max_trajectory_restarts": 1000  # Maximum number of trajectory restarts allowed
        }
) 
exp_trajectory_length.show_experiment_df()

##### 3. Amount of Trajectories Required for Stable Inference

**Objective.**
To determine how many independent trajectories are required to obtain statistically stable reconstructions.

**Experimental design.**

- Sampling frequency and trajectory length are fixed to values identified as near-optimal in previous experiments.
- The number of trajectories per dataset is gradually increased - from 10 to 100 by 10.
- For each setting, multiple (30) independent repetitions are performed to obtain convergent distribution .

**Evaluation.**

- Reconstruction accuracy is summarized using distributions (score functions).
- Stability is assessed by observing convergence of accuracy metrics as the number of trajectories increases.
- No classical parametric hypothesis test is assumed; instead, convergence trends is reported.
<!-- Nie znalazłem żadnego sensownego -->

**Rationale.**
Due to the randomness of Boolean functions and initial states, averaging over multiple networks is necessary to separate systematic effects from instance-specific variability.


In [ ]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    # paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',
    # experiment values
    num_nodes=[4, 6],
    update_mode=["synchronous"], # , "asynchronous"
    trajectory_length=list(range(50, 61, 10)),
    n_trajectories=[50],
    sampling_frequency=[1],
    score_functions=["MDL", "BDE"],
    # TODO - add metrics options 

    n_parents_per_node=[[1, 2, 3, 4]],
    # amount of samples per experiment 
    n_repetitions=2,
    simulate_trajectories_to_csv_kwargs = {
            # "sampling_frequency": 1,
            "target_attractor_ratio": 0.4,  # Approximate fraction of trajectory in attractor (0-1)
            "tolerance": 0.3,               # Allowed deviation from the calculated entrance step (0-1)
            "max_iter": 100,                 # Maximum attempts to generate a valid state per step before restarting
            "max_trajectory_restarts": 1000  # Maximum number of trajectory restarts allowed
        }
) 
exp_trajectory_length.show_experiment_df()

#### Analysis

In [ ]:
#TODO - tutaj zrobimy analize wyników (więc ploty będziemy wstawiać, i )

## Part 2

### 1. Choose validated Boolean network
#### Task
our task is to consider a validated Boolean network model of a real-life biological mechanism. To this end, select a Boolean network 2 model of your choice from the ‘models’ subfolder of the Biodivine repository, available at https://github.com/sybila/biodivine-boolean-models, with the number of nodes (variables) not exceeding 16.‡. 

In [ ]:
# TODO - wybrac sieć

# TODO - 

### 2. Generate dataset,
#### Task
Using the insights gained from the first part of the project,
generate an appropriate dataset for the network inference task. 




In [ ]:
# TODO - tworzymy dataset (na bazie wniosków z poprzedniego)
# TODO - tutaj bierzemy wszystkie datasety, jakie wnioski udało nam się ustalić do nich

### 3. Reconstruct the network with BNFinder2 
#### Task
Reconstruct the network structure with BNFinder2, applying a scoring function chosen based on your previous
experience. Evaluate the accuracy of the reconstruction.

In [ ]:
# TODO zliczenie jakości dopasowania